In [1]:
import os
from PIL import Image
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import DataLoader 
from torch.utils.data import Dataset
import torchvision.transforms as transforms

# Create Dataset Class

In [2]:
class SignLanguageDigits(Dataset):
    
    def __init__(self, root_dir, shape, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform or transforms.Compose([
            transforms.Resize(shape),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        self.image_paths = []
        self.labels = []
        
        # Split data into train/test (80/20)
        for label in range(10):
            digit_dir = os.path.join(root_dir, str(label))
            images = [img for img in os.listdir(digit_dir) if img.endswith('.JPG')]
            split_idx = int(0.8 * len(images))   
            
            if train:
                images = images[:split_idx]
            else:
                images = images[split_idx:]
                
            for img_name in images:
                self.image_paths.append(os.path.join(digit_dir, img_name))
                self.labels.append(label)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')  # Ensure 3 channels
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Create and Train NN

In [9]:
batch_size = 64


# Replace the MNIST dataset loading with:
train_dataset = SignLanguageDigits(
    root_dir='Sign-Language-Digits-Dataset/Dataset',
    shape=(64, 64),  # Resize to 64x64
    train=True
)

test_dataset = SignLanguageDigits(
    root_dir='Sign-Language-Digits-Dataset/Dataset',
    shape=(64, 64),
    train=False
)

train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)



class NN(nn.Module):

    def __init__(self, input_size, num_classes): # constructor of NN with its attributes

        super(NN, self).__init__() # calling constructor of base class  

        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)

        # callable objects

    def forward(self, x):  # we must provid imp of forward () of nn.Module in our subclass

        x = F.relu(self.fc1(x)) # //can do F.softmax(self.fc1(x))
 
        x = F.relu(self.fc2(x))

        x = self.fc3(x)  #         x = F.softmax(self.fc3(x), dim=1)

        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 12288 # 3x64x64 = 12,288 size of sign images (RGB)

num_classes = 10

learning_rate = 0.001

num_epochs = 10


# create NN object and move it to device

model = NN(input_size=input_size, num_classes=num_classes).to(device)


criterion = nn.CrossEntropyLoss()
 

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

 

for epoch in range(num_epochs):

    print(f"Epoch: {epoch}")

    for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible

        data = data.to(device=device)

        targets = targets.to(device=device)

        data = data.reshape(data.shape[0], -1) #[64,3x64x64]=[64, 12288]

        # forward propagation

        scores = model(data) #automatically call the forward method,


        loss = criterion(scores, targets) # compute cost/loss on 64 example

        # zero previous gradients

        optimizer.zero_grad()

       
        # back-propagation

        loss.backward()

        # gradient descent or adam step

        optimizer.step()


Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9


# Test Accuracy

In [10]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # 1. our model deactivates all the layers (eg.batch normalization/dropout)
    with torch.no_grad(): #2.  not make computational graph
        for x, y in loader:
            #print (x.shape)
            x = x.to(device=device)
            y = y.to(device=device)
           
            x = x.reshape(x.shape[0], -1)
            print(x.shape)
            #print (y.shape)
            
            scores = model(x)
            print(scores.shape)
                      
            _, predictions = scores.max(1) #. it return max value and its index, 1 mean see column-wise 
            
            num_correct += (predictions == y).sum() # compare prediction with y, if equal sum them to count the number of same values
            num_samples += predictions.size(0)  #64, get no of samples
            break  # just to see the results for a single patch
        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct) / float(num_samples) * 100:.2f}"
        )


        
print ("Test accuracy: ")
check_accuracy(test_loader, model)

Test accuracy: 
torch.Size([64, 12288])
torch.Size([64, 10])
Got 55 / 64 with accuracy 85.94
